In [0]:
%python
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType
from pyspark.sql.functions import lit, col, when

# Update the final status of the run
def finalize_run(status_table, job_run_id):
    final_status = "Done"
    end_time = datetime.now() 
    status_table = status_table.withColumn("end_time", 
                                           when(col("run_id") == job_run_id, lit(end_time)).otherwise(col("end_time")))
    status_table = status_table.withColumn("status", 
                                           when(col("run_id") == job_run_id, lit(final_status)).otherwise(col("status")))
    status_table = status_table.withColumn("run_duration", 
                                           when(col("run_id") == job_run_id, 
                                                (col("end_time").cast("long") - col("start_time").cast("long"))).otherwise(col("run_duration")))
    return status_table

status_table = spark.table("default.status")
job_run_id = dbutils.widgets.get("run_id")

# Finalize the run
status_table = finalize_run(status_table, job_run_id)

#display(status_table)

# Step 5: Store the updated DataFrame as a new table
status_table.write.mode("overwrite").saveAsTable("default.status")